# 多个输入和输出通道

- 彩色图像可能有RGB三个通道
- 转换为灰度会丢失信息
<img src="./pic/lena.jpg" width=300 height=300>


## 1. 多个输入通道，单个输出通道

- 输入不再是一个矩阵，而是一个三维的张量（tensor）
- 每个通道都有一个卷积核，结果是所有通道卷积结果的和
<img src="./pic/多个输入通道.PNG" width=500 height=500>
- 公式：
<img src="./pic/多个输入通道公式.PNG" width=300 height=300>
- 最后的输出是**单通道**


## 2. 多个输出通道

- 不管多少个输入通道，只得到1个输出通道不够。
- 增加通道数，使得能够匹配的模式种类增加。相当于同样一个像素，能表达的模式种类，增加了。
<img src="./pic/多个输出通道.PNG" width=400 height=400>
- c<sub>i</sub>个输入通道，每个输入通道输出c<sub>o</sub>种卷积核，共有c<sub>i</sub> x c<sub>o</sub>种卷积核。
    - c<sub>i</sub>：输入通道的（卷积核）层数，
    - c<sub>o</sub>：输出通道的（卷积核）层数。（每一个输出通道为了识别某一个特定的模式：提取出不同的特征）
    - 两者无相关性 
- 对每一个输入X（黄线），拿出对应的一个输出通道的核（黄线），得到一个对应的输出通道（黄线）。然后再把输出通道一一做运算，再concate，得到多个输出通道。


## 多个输入和多个输出通道

- 每个输出通道可以识别特定模式
    - 上篇笔记的骆驼，三个输出通道分别提取边缘特征，锐化特征和高斯特征
    - 本例中，输出通道数为6：绿色点，红色点，渐变，斜纹理，竖纹理，模糊

<img src="./pic/多个输入和多个输出通道.PNG" width=400 height=400>

- 当前层的输入通道把前一层的6种输出通道，组合起来并加权，得到一个组合的模式识别
    - 比如纹理比较重要，则纹理权重大，颜色权重小。

## 1 x 1 卷积层

- 不会识别空间信息
    - 因为每次只识别1个像素，不会看这个像素周围的像素
- 三个通道压缩成一个通道：融合不同通道的信息
<img src="./pic/1x1卷积.PNG" width=400 height=400>

- 浅蓝色：三个输入维，输出为第0通道(output channel 0)
- 深蓝色：三个输入维，输出为第1通道(output channel 1)
- 将 1×1 卷积层看作是在每个像素位置应用的全连接层，以 c<sub>i</sub> 个输入值转换为 c<sub>o</sub> 个输出值
    - 相当于输入形状为n<sub>h</sub>n<sub>w</sub> x c<sub>i</sub>，权重为c<sub>i</sub> x c<sub>o</sub>的全连接层
- 因为这仍然是一个卷积层，所以跨像素的权重是一致的



## 二维卷积层

- 输出m的size取决于输入k的size、padding和strides
- 一共有 c<sub>i</sub> x c<sub>o</sub> 个卷积核 每个卷积核都有一个偏差
<img src="./pic/二维卷积层维度.PNG" width=400 height=400>
- 1GFLOP x 10层 x 1M样本 = 10Pflops。

## 总结

- 输出通道数是卷积层的超参数（输入是前一层的超参数，不是当前层）
- 每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果
- 每个输出通道有独立的三维卷积核，所以最后的卷积核是一个四维的张量

In [ ]:
def corr2d(X, K):    # X为输入，K为核矩阵
    """计算二维互相关运算"""
    h, w = K.shape    # 返回行数和列数
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    # 输入的高度：X.shape[0] - h + 1
    # 输入的宽度：X.shape[1] - w + 1
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()    # 哈达玛积(不是数学上传统的矩阵乘积)，再求和
    return Y

# d2l
def corr2d(X, K):
    """Compute 2D cross-correlation.

    Defined in :numref:`sec_conv_layer`"""
    h, w = K.shape
    Y = d2l.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = d2l.reduce_sum((X[i: i + h, j: j + w] * K))
    return Y

# 多输入多输出通道的代码实现
## 1. 实现多输入互相关运算

In [1]:
import torch
from d2l import torch as d2l


def corr2d_multi_in(X, K):
    # 对最外层输入通道做遍历
    # 每一次遍历，拿出输入通道的矩阵
    # 做互相关运算，最后对元素求和
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起

# zip函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表。

In [19]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

X, X.shape, K, K.shape

(tensor([[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]],
 
         [[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]),
 torch.Size([2, 3, 3]),
 tensor([[[0., 1.],
          [2., 3.]],
 
         [[1., 2.],
          [3., 4.]]]),
 torch.Size([2, 2, 2]))

In [20]:
Y = corr2d_multi_in(X, K)
Y, Y.shape

(tensor([[ 56.,  72.],
         [104., 120.]]),
 torch.Size([2, 2]))

## 2. 计算多个通道的输出的互相关函数
- 上一步是，两层输入一层输出，两层输入和对的卷积核做完卷积运算后求和，变成一层输出
- 这一步是，堆叠了三层上一步的操作，就是三组卷积核，一组两个卷积核，所以输出的就是三组卷积核和输入分别做上一步的运算

In [23]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    # 对K，先拿出k，k为3D tensor
    # k与X先做互相关运算，得到一个2D 矩阵
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
K = torch.stack((K, K + 1, K + 2), 0)    # 三个卷积核，每个卷积核有两个通道，每个通道是2*2的矩阵
K, K.shape

(tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]),
 torch.Size([3, 2, 2, 2]))

In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 1×1  卷积层（等价于一个全连接层）

In [24]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))    # h * w：把高和宽拉成一条向量（c_i变成了batch_size）
    K = K.reshape((c_o, c_i))      # c_o * c_i * 1 * 1，拿掉最后两个维度
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)    # 全连接层
Y2 = corr2d_multi_in_out(X, K)        # 卷积层
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6    # Y1应该等于Y2

每个通道去识别特定的模式，所以通道之间参数不相同
每个通道的卷积核数值是不一样，但卷积核大小size是一样的